# 多項ロジスティック回帰

このノートブックでは、多項ロジスティック回帰（ソフトマックス回帰）を実装します。

## 学習目標
- ソフトマックス関数の理解と実装
- 多項ロジスティック回帰の理論
- 交差エントロピー損失の計算
- 勾配降下法による最適化


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")


## 1. ソフトマックス関数の実装


In [ ]:
def softmax(z):
    """
    ソフトマックス関数の実装
    
    Parameters:
    z: 線形結合の結果 (n_samples, n_classes)
    
    Returns:
    softmax(z): ソフトマックス関数の値
    """
    # 数値的安定性のため、最大値を引く
    z_shifted = z - np.max(z, axis=1, keepdims=True)
    exp_z = np.exp(z_shifted)
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# ソフトマックス関数の可視化
z_example = np.array([[1, 2, 3], [0, 0, 0], [-1, 0, 1]])
softmax_example = softmax(z_example)

print("=== ソフトマックス関数の例 ===")
print(f"入力 z: {z_example}")
print(f"ソフトマックス出力: {softmax_example}")
print(f"各行の合計: {softmax_example.sum(axis=1)}")

# ソフトマックス関数の特性を可視化
z_range = np.linspace(-10, 10, 100)
z_2d = np.column_stack([z_range, np.zeros_like(z_range), -z_range])
softmax_2d = softmax(z_2d)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(z_range, softmax_2d[:, 0], 'r-', label='Class 0', linewidth=2)
plt.plot(z_range, softmax_2d[:, 1], 'g-', label='Class 1', linewidth=2)
plt.plot(z_range, softmax_2d[:, 2], 'b-', label='Class 2', linewidth=2)
plt.xlabel('z[0]')
plt.ylabel('Softmax Probability')
plt.title('Softmax Function for 3 Classes')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(z_range, softmax_2d.sum(axis=1), 'k-', linewidth=2)
plt.xlabel('z[0]')
plt.ylabel('Sum of Probabilities')
plt.title('Sum of Softmax Probabilities (should be 1)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 2. 多項ロジスティック回帰の実装


In [ ]:
class MultinomialLogisticRegression:
    """
    多項ロジスティック回帰の実装
    """
    
    def __init__(self, learning_rate=0.01, max_iterations=1000, tolerance=1e-6):
        """
        パラメータの初期化
        
        Parameters:
        learning_rate: 学習率
        max_iterations: 最大反復回数
        tolerance: 収束判定の閾値
        """
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        self.tolerance = tolerance
        self.weights = None
        self.bias = None
        self.cost_history = []
        self.classes = None
    
    def fit(self, X, y):
        """
        モデルの訓練
        
        Parameters:
        X: 特徴量
        y: ラベル
        """
        # クラスの取得
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_samples, n_features = X.shape
        
        # パラメータの初期化
        self.weights = np.zeros((n_classes, n_features))
        self.bias = np.zeros(n_classes)
        
        # 勾配降下法
        for i in range(self.max_iterations):
            # 線形結合の計算
            z = np.dot(X, self.weights.T) + self.bias
            
            # ソフトマックス関数の適用
            probabilities = softmax(z)
            
            # 損失の計算
            cost = self._cross_entropy_loss(y, probabilities)
            self.cost_history.append(cost)
            
            # 勾配の計算
            dw, db = self._compute_gradients(X, y, probabilities)
            
            # パラメータの更新
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
            
            # 収束判定
            if i > 0 and abs(self.cost_history[-1] - self.cost_history[-2]) < self.tolerance:
                print(f"収束しました。反復回数: {i+1}")
                break
        
        return self
    
    def _cross_entropy_loss(self, y, probabilities):
        """
        交差エントロピー損失の計算
        
        Parameters:
        y: 実際のラベル
        probabilities: 予測確率
        
        Returns:
        loss: 交差エントロピー損失
        """
        # ワンホットエンコーディング
        y_one_hot = np.eye(len(self.classes))[y]
        
        # 数値的安定性のため、確率をクリップ
        probabilities = np.clip(probabilities, 1e-15, 1 - 1e-15)
        
        # 交差エントロピー損失の計算
        loss = -np.mean(np.sum(y_one_hot * np.log(probabilities), axis=1))
        return loss
    
    def _compute_gradients(self, X, y, probabilities):
        """
        勾配の計算
        
        Parameters:
        X: 特徴量
        y: ラベル
        probabilities: 予測確率
        
        Returns:
        dw: 重みの勾配
        db: バイアスの勾配
        """
        n_samples = X.shape[0]
        n_classes = len(self.classes)
        
        # ワンホットエンコーディング
        y_one_hot = np.eye(n_classes)[y]
        
        # 勾配の計算
        dw = np.dot((probabilities - y_one_hot).T, X) / n_samples
        db = np.mean(probabilities - y_one_hot, axis=0)
        
        return dw, db
    
    def predict_proba(self, X):
        """
        確率の予測
        
        Parameters:
        X: 特徴量
        
        Returns:
        probabilities: 予測確率
        """
        z = np.dot(X, self.weights.T) + self.bias
        return softmax(z)
    
    def predict(self, X):
        """
        クラスの予測
        
        Parameters:
        X: 特徴量
        
        Returns:
        predictions: 予測クラス
        """
        probabilities = self.predict_proba(X)
        return self.classes[np.argmax(probabilities, axis=1)]
    
    def score(self, X, y):
        """
        精度の計算
        
        Parameters:
        X: 特徴量
        y: ラベル
        
        Returns:
        accuracy: 精度
        """
        predictions = self.predict(X)
        return accuracy_score(y, predictions)

# データの準備
X, y = make_classification(
    n_samples=1000,
    n_features=2,
    n_redundant=0,
    n_informative=2,
    n_clusters_per_class=1,
    n_classes=3,
    random_state=42
)

# データの分割と標準化
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("=== データセットの情報 ===")
print(f"訓練データの形状: {X_train_scaled.shape}")
print(f"テストデータの形状: {X_test_scaled.shape}")
print(f"クラス数: {len(np.unique(y))}")
print(f"クラスの分布: {np.bincount(y)}")


## 3. モデルの訓練と評価


In [ ]:
# モデルの訓練
model = MultinomialLogisticRegression(learning_rate=0.1, max_iterations=1000)
model.fit(X_train_scaled, y_train)

# 損失の履歴を可視化
plt.figure(figsize=(10, 6))
plt.plot(model.cost_history)
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.title('Cost History During Training')
plt.grid(True, alpha=0.3)
plt.show()

print(f"最終的な損失: {model.cost_history[-1]:.4f}")
print(f"重みの形状: {model.weights.shape}")
print(f"バイアスの形状: {model.bias.shape}")

# 予測の実行
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)

# 精度の計算
accuracy = model.score(X_test_scaled, y_test)

print(f"\n=== モデルの評価結果 ===")
print(f"Accuracy: {accuracy:.4f}")

# 混同行列の可視化
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Class 0', 'Class 1', 'Class 2'], 
            yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# 分類レポート
plt.subplot(1, 2, 2)
plt.text(0.1, 0.9, 'Classification Report:', fontsize=14, fontweight='bold', transform=plt.gca().transAxes)
plt.text(0.1, 0.8, classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1', 'Class 2']), 
         fontsize=10, transform=plt.gca().transAxes, verticalalignment='top')
plt.axis('off')
plt.title('Classification Report')

plt.tight_layout()
plt.show()


## 4. scikit-learnとの比較


In [ ]:
# scikit-learnの多項ロジスティック回帰
sklearn_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42, max_iter=1000)
sklearn_model.fit(X_train_scaled, y_train)

# 予測
sklearn_pred = sklearn_model.predict(X_test_scaled)
sklearn_proba = sklearn_model.predict_proba(X_test_scaled)

# 精度の比較
sklearn_accuracy = sklearn_model.score(X_test_scaled, y_test)

print("=== scikit-learnとの比較 ===")
print(f"自作モデルの精度: {accuracy:.4f}")
print(f"scikit-learnの精度: {sklearn_accuracy:.4f}")
print(f"精度の差: {abs(accuracy - sklearn_accuracy):.4f}")

# 係数の比較
print(f"\n=== 係数の比較 ===")
print(f"自作モデルの重み: {model.weights}")
print(f"scikit-learnの重み: {sklearn_model.coef_}")
print(f"自作モデルのバイアス: {model.bias}")
print(f"scikit-learnのバイアス: {sklearn_model.intercept_}")

# 予測確率の比較
plt.figure(figsize=(15, 5))

for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.scatter(y_pred_proba[:, i], sklearn_proba[:, i], alpha=0.6)
    plt.plot([0, 1], [0, 1], 'r--', alpha=0.8)
    plt.xlabel('自作モデルの予測確率')
    plt.ylabel('scikit-learnの予測確率')
    plt.title(f'Class {i} Probability Comparison')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. 演習問題

### 演習1: 学習率の影響
異なる学習率でモデルを訓練し、収束の様子を比較してみましょう。

### 演習2: 正則化の実装
L2正則化を追加して、過学習を防ぐ実装を作成してみましょう。

### 演習3: クラス数の影響
異なるクラス数でデータセットを生成し、多項ロジスティック回帰の性能を確認してみましょう。


## まとめ

このノートブックでは、多項ロジスティック回帰（ソフトマックス回帰）を実装しました。

**学習した内容**：
- ソフトマックス関数の実装と特性
- 多項ロジスティック回帰の理論
- 交差エントロピー損失の計算
- 勾配降下法による最適化
- scikit-learnとの比較

**重要なポイント**：
- ソフトマックス関数は確率の正規化を自動的に行う
- 数値的安定性に注意が必要
- 多項ロジスティック回帰は1つのモデルで全クラスを同時に扱う
- 勾配の計算が複雑になる

**次のステップ**：
- 正則化の実装
- より高度な最適化手法の学習
- 深層学習への応用
